# Импорт библиотек

In [14]:
# Импорт библиотеки для работы с json-файлами
import json

# импортируем библиотеку numpy и pandas
import pandas as pd
import numpy as np

# импортируем библиотеку для работы с картами
from keplergl import KeplerGl

# импортируем библиотеку для вычисление высоты
import srtm

CGN Airport elevation (meters): 423


In [17]:
# Использовать для вычисления высоты 
# elevation_data = srtm.get_data()
# print('CGN Airport elevation (meters):', elevation_data.get_elevation(52.274269, 104.274618))

## Шпаргалка

<img src="https://ночная.москва/wp-content/uploads/2021/03/geograficheskie-koordinaty-moskvy-rossiya.png" width="50%" height="50%">

## Координаты вершин полигона г. Иркутск

In [61]:
# Границы полигона Иркутска
up_left = [52.446093, 103.964495]
up_right = [52.446093, 104.802521]

down_left = [52.013882, 103.964495]
down_right = [52.013882, 104.802521]

### Создание `pd.DataFrame` - полигона г. Иркутск 

In [ ]:
coordinates = []

coordinates.append(up_left)
coordinates.append(up_right)
coordinates.append(down_left)
coordinates.append(down_right)

headers = ['latitude', 'longitude']

df_coordinates = pd.DataFrame(coordinates, columns = headers)
df_coordinates

## Генерация множества координат

In [176]:
generated_list_of_latitude = np.arange(down_left[0], up_left[0], 0.001)

generated_list_of_longitude = np.arange(up_left[1], up_right[1], 0.001)

## Создание `pd.DataFrame` с "заполнением" полигона г. Иркутск

In [180]:
%%time

# Создание пустого датафрейма
df_append = pd.DataFrame()

# Цикл по все показателям широты
for lat in range(len(generated_list_of_latitude)):
    # Создание словаря с данными. К кажд
    data = {
        'latitude': generated_list_of_latitude[lat],
        'longitude': generated_list_of_longitude
    }
    
    df = pd.DataFrame(data)
    
    df_append = df_append.append(df)
    
df_append.shape

CPU times: user 272 ms, sys: 28.6 ms, total: 301 ms
Wall time: 305 ms


(363287, 2)

In [148]:
df_append.latitude.unique()

array([52.013882, 52.013891, 52.0139  , 52.013909, 52.013918, 52.013927,
       52.013936, 52.013945, 52.013954, 52.013963])

,latitude,longitude
0,52.446093,103.964495
1,52.446093,104.802521
2,52.013882,103.964495
3,52.013882,104.802521


In [181]:
with open('config.json') as f:
    config = json.load(f)

map_1 = KeplerGl()
map_1.config = config

map_1.add_data(data=df_append, name='9673oreu')

map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'prj4oj', 'type': '…